# Janelando amostras do SESA

Antes de usar o ReSpeaker para gravar amostras de X ms, vou janelas algumas amostras do SESA para depois medir o tempo que a Raspberry leva para normalizar as features da nova amostra e classificar.

In [1]:
import os
import librosa
import shutil
import soundfile
import subprocess
import numpy as np
from IPython.display import Audio

/home/pi/.pyenv/versions/3.6.1/envs/venv1/lib/python3.6/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


## Parâmetros iniciais e funções

In [2]:
freqAmostragem = 16000
frameLength    = int(0.200 * freqAmostragem)
overlapLength  = int(frameLength/2)

In [3]:
def fazerJanelamento(sinal, frameLength, overlapLength):
    return librosa.util.frame(sinal, frame_length=frameLength, hop_length=overlapLength).T

## Ajustando os diretórios

Definindo onde estão os áudios completos e pra onde irão as amostras janeladas.

In [4]:
dirOrigem  = "/home/pi/GravacoesReSpeaker/exemplos_amostras_SESA_v2_16kHz_16bits/"
dirDestino = dirOrigem + "exemplos_janelados_200ms/"

Se o diretório de destino já existir, eu vou excluir tudo e criar do zero.

In [5]:
if os.path.isdir(dirDestino) == True:
    shutil.rmtree(dirDestino)
    print("O diretório de destino já existia. Todo seu conteúdo foi excluido.")

# JA VOU PEGAR OS ARQUIVOS COMPLETOS DA ORIGEM AGR PRA VIR SO OS WAVS
arquivosCompletos = os.listdir(dirOrigem)

# AGR SIM EU CRIO A PASTA DE DESTINO
os.mkdir(dirDestino)

O diretório de destino já existia. Todo seu conteúdo foi excluido.


## Garantindo que a função funciona como desejado

Na documentação fala que vc pode enviar o parâmetro "axis" pra função "frame" do librosa. A opção default é -1, que NÃO faz o que nós desejamos. Por isso na função ali em cima eu coloquei ".T" pra pegar a transposta. Vamos verificar que é isso mesmo.

In [6]:
sinalOriginal, freqAmostragem = librosa.load(dirOrigem+arquivosCompletos[1], sr=None, mono=True)
janelas = fazerJanelamento(sinalOriginal, frameLength, overlapLength)
print("O resultado deve gerar", int((len(sinalOriginal)/frameLength)*(frameLength/overlapLength))-1, "janelas de", frameLength, "amostras cada.")
print("Shape do resultado:", janelas.shape, "\n")
print("Sinal original:")
Audio(data=sinalOriginal, rate=freqAmostragem)

O resultado deve gerar 121 janelas de 3200 amostras cada.
Shape do resultado: (121, 3200) 

Sinal original:


Como usamos sempre 50% de overlap, vamos pulando de uma em uma janela pra concatenar as janelas que formam o sinal original.

In [7]:
# JANELAS PARES
reconstrucao = []
for i in range(0, len(janelas), 2):
    reconstrucao.extend(janelas[i])
Audio(data=reconstrucao, rate=freqAmostragem)

In [8]:
# JANELAS IMPARES
reconstrucao = []
for i in range(1, len(janelas), 2):
    reconstrucao.extend(janelas[i])
Audio(data=reconstrucao, rate=freqAmostragem)

Só por curiosidade, vamos ver como fica se concatenarmos todas as janelas, inclusive as de sobreposição hehehe.

In [9]:
# TODAS AS JANELAS JUNTAS (BAGUNCA)
reconstrucao = []
for i in range(1, len(janelas)):
    reconstrucao.extend(janelas[i])
Audio(data=reconstrucao, rate=freqAmostragem)

## Janelando os áudios

In [10]:
for arquivoAtual in arquivosCompletos:
    
    # JANELANDO O ARQUIVO ATUAL
    sinalOriginal, freqAmostragem = librosa.load(dirOrigem+arquivoAtual, sr=None, mono=True)
    janelas = fazerJanelamento(sinalOriginal, frameLength, overlapLength)
    
    print("O resultado deve gerar", int((len(sinalOriginal)/frameLength)*(frameLength/overlapLength))-1, "janelas de", frameLength, "amostras cada.")
    print("Shape do resultado:", janelas.shape, "\n")
    
    # SALVANDO CADA JANELA NO DIRETORIO DE DESTINO
    for i, janelaAtual in enumerate(janelas):
        novoNome = arquivoAtual[:-4] + "_janela_" + str(i) + ".wav"
        librosa.output.write_wav(dirDestino + novoNome, janelaAtual, freqAmostragem)

O resultado deve gerar 31 janelas de 3200 amostras cada.
Shape do resultado: (31, 3200) 

O resultado deve gerar 121 janelas de 3200 amostras cada.
Shape do resultado: (121, 3200) 

O resultado deve gerar 9 janelas de 3200 amostras cada.
Shape do resultado: (9, 3200) 

O resultado deve gerar 45 janelas de 3200 amostras cada.
Shape do resultado: (45, 3200) 



## Verificando os arquivos gerados

O librosa NÃO PRESERVA A PROFUNDIDADE DE BITS. Portanto, vamos verificar como está a situação e, conforme for, vamos usar o SoX pra manter as coisas em 16 bits.

In [11]:
arquivosDestino = os.listdir(dirDestino)
print("Total de arquivos gerados:", len(arquivosDestino))

Total de arquivos gerados: 206


In [12]:
arquivoSF = soundfile.SoundFile(dirDestino + arquivosDestino[0])
print('Frequência de amostragem:', arquivoSF.samplerate)
print('Canais:', arquivoSF.channels)
print('Profundidade de bits:', arquivoSF.subtype)

Frequência de amostragem: 16000
Canais: 1
Profundidade de bits: FLOAT


Hmmm, como suspeitava... Vamos ter que passar esse "FLOAT" ai pra "PCM_16".

In [13]:
for arquivoAtual in arquivosDestino:
    subprocess.run(["sox", dirDestino+arquivoAtual, "-b 16", dirDestino+arquivoAtual])

Agora vamos ver se tá tudo em 16 bits msm...

In [14]:
for arquivoAtual in arquivosDestino[100:110]:
    arquivoSF = soundfile.SoundFile(dirDestino + arquivoAtual)
    print("Arquivo:", arquivoAtual)
    print("Frequência de amostragem:", arquivoSF.samplerate)
    print("Canais:", arquivoSF.channels)
    print("Profundidade de bits:", arquivoSF.subtype, "\n")

Arquivo: explosion_000_janela_4.wav
Frequência de amostragem: 16000
Canais: 1
Profundidade de bits: PCM_16 

Arquivo: siren_000_janela_84.wav
Frequência de amostragem: 16000
Canais: 1
Profundidade de bits: PCM_16 

Arquivo: siren_000_janela_11.wav
Frequência de amostragem: 16000
Canais: 1
Profundidade de bits: PCM_16 

Arquivo: siren_000_janela_79.wav
Frequência de amostragem: 16000
Canais: 1
Profundidade de bits: PCM_16 

Arquivo: siren_000_janela_37.wav
Frequência de amostragem: 16000
Canais: 1
Profundidade de bits: PCM_16 

Arquivo: gunshot_000_janela_4.wav
Frequência de amostragem: 16000
Canais: 1
Profundidade de bits: PCM_16 

Arquivo: siren_000_janela_14.wav
Frequência de amostragem: 16000
Canais: 1
Profundidade de bits: PCM_16 

Arquivo: explosion_000_janela_35.wav
Frequência de amostragem: 16000
Canais: 1
Profundidade de bits: PCM_16 

Arquivo: explosion_000_janela_42.wav
Frequência de amostragem: 16000
Canais: 1
Profundidade de bits: PCM_16 

Arquivo: explosion_000_janela_9.wa